## Sample Workflow :

- ### 1: data (prepare and load)
- ### 2: bulid model
- ### 3: fitting the model to data (Training)
- ### 4: making prediction and evaluating a model
- ### 5: saving amd loading a model
- ### 6: putting it all togother


In [1]:
import torch 
from torch import nn # nn contains all pytorch building blocks for neural networks
import matplotlib.pyplot as plt

- ### Data: Preparig and loading

#### data can be almost anything...
* Tabular data
* Images
* Videos
* Audio
* DNA
* Text

Machine learning is a game of two parts:
1. Get data into numerical representation.
2. Build Proper Model For learning that.

### Use linear regression formula to make a straight line with **known parameters**

In [2]:
weight = 0.7
bias = 0.3

In [3]:
# create data
start = 0
end = 1
step = 0.02

X = torch.arange(start, end, step)
X[:5]

tensor([0.0000, 0.0200, 0.0400, 0.0600, 0.0800])

In [4]:
X.shape

torch.Size([50])

In [5]:
X = X.unsqueeze(dim=1)
X[:5]

tensor([[0.0000],
        [0.0200],
        [0.0400],
        [0.0600],
        [0.0800]])

In [6]:
X.shape

torch.Size([50, 1])

In [7]:
y = X * weight + bias

In [8]:
X[:5], y[:5]

(tensor([[0.0000],
         [0.0200],
         [0.0400],
         [0.0600],
         [0.0800]]),
 tensor([[0.3000],
         [0.3140],
         [0.3280],
         [0.3420],
         [0.3560]]))

In [9]:
### spliting data into train and test sets ###

In [10]:
train_split = int(0.8 * len(X))

X_train, y_train = X[:train_split], y[:train_split]
X_test, y_test = X[train_split:], y[train_split:]

len(X_train), len(y_train), len(X_test), len(y_test)

(40, 40, 10, 10)

In [11]:
### Visualize our data ###

In [12]:
def plot_prediction(train_data=X_train, train_labels=y_train,
                    test_data=X_test, test_labels=y_test, 
                    predictions=None):
    """
    plot training and test data and compare them with predictions
    
    """
    
    plt.figure(figsize=(10, 7))
    # trainind data
    plt.scatter(train_data, train_labels, c='b', s=4, label='Training data')
    # test data
    plt.scatter(test_data, test_labels, c='r', s=4, label='Testing data')
    
    # plot the predictions if they exists:
    if predictions is not None:
        plt.scatter(test_data, predictions, c='g', s=4, label='Predictions')
         
            
    plt.legend();
    
    

In [13]:
# plot_prediction();

- ### Building first Model

our model :
- Start with random values
- Look at tgraining data
- Adjust random values to better represent(or get closer to) the ideal values

How to adjust?
- 1. Gradient decent
- 2. BackPropagation

In [14]:
### Create Linear Regressin Python Classes ###

In [15]:
class LinearRegressionModel(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.weights = nn.Parameter(torch.rand(1, dtype=torch.float, requires_grad=True))
        self.bias = nn.Parameter(torch.rand(1, dtype=torch.float, requires_grad=True))
        
    # Forward method to define computation in the model :
    def forward(self, x:torch.Tensor) -> torch.Tensor: # x is the input data
        return self.weights * x + self.bias


### PyTorch model building essentials :

- **torch.nn** --> contains all of the buildings for computational graphs (Neurak Networks)

- **torch.nn.Parameter** --> (what paramater should our model try and learn, often a pytorch layer from torch.nn will set this for us)

- **torch.nn.Module** --> The base class for all neural network modules, if cubclass it, forward() should be overwrited.

- **torch.optim** --> this is where the optimizers in pytorch live, they will help with gradient decent

- **def forward()** --> feed forward in network.

In [16]:
### checking the contents of our PyTorch model :

# create a random seed
torch.manual_seed(42)

model_0 = LinearRegressionModel()
list(model_0.parameters())

[Parameter containing:
 tensor([0.8823], requires_grad=True),
 Parameter containing:
 tensor([0.9150], requires_grad=True)]

In [17]:
# list named parameter
model_0.state_dict()

OrderedDict([('weights', tensor([0.8823])), ('bias', tensor([0.9150]))])

In [18]:
X_test, y_test

(tensor([[0.8000],
         [0.8200],
         [0.8400],
         [0.8600],
         [0.8800],
         [0.9000],
         [0.9200],
         [0.9400],
         [0.9600],
         [0.9800]]),
 tensor([[0.8600],
         [0.8740],
         [0.8880],
         [0.9020],
         [0.9160],
         [0.9300],
         [0.9440],
         [0.9580],
         [0.9720],
         [0.9860]]))

In [19]:
# make predictions with model:
with torch.inference_mode():
     y_preds=model_0(X_test)
        
y_preds


## equivalent :
# with torch.no_grad():
#     y_preds=model_0(X_test)
    
# y_preds

tensor([[1.6208],
        [1.6385],
        [1.6561],
        [1.6738],
        [1.6914],
        [1.7090],
        [1.7267],
        [1.7443],
        [1.7620],
        [1.7796]])

### Train Model

start with random parameters and learn from data to get better representation.

One way to measure how poor model prediction are is to use **Loss Function(Cost Function/ Criterion)**.


Things we need to train:
- Loss Function
- Optimizer (adjust parameter base on loss to improve the Loss Function)

In pytorch we need :
- Training Loop
- Testing Loop



In [20]:
# Setup a loss function
loss_fn = nn.L1Loss()

# Setup an optimizer
optimizer = torch.optim.SGD(params=model_0.parameters(), lr=0.01)

### Building a Training Loop(and Testing Loop) in pytorch

Things we need in training loop:

0. Loop through the data
1. Forward pass(this involves data moving through our models `forward()` functions)
2. Calculate the Loss
3. Optimizer zero grad
4. Loss Backward - move backwards through the networks **Backpropagation**
5. Optimizer Step - use optimizer to adjust model parameters. **Gradient Decent**

In [21]:
# weight = 0.7
# bias = 0.3

model_0.state_dict()

OrderedDict([('weights', tensor([0.8823])), ('bias', tensor([0.9150]))])

In [22]:
epochs = 100

epoch_count = []
train_loss_values = []
test_loss_values = []

for epoch in range(epochs):
    #set the model to training mode
    model_0.train() # track requires gradient
    
    # 1. Forward pass
    y_pred = model_0(X_train)
    
    # 2. Loss
    loss = loss_fn(y_pred, y_train)
    
    # 3. optimizer zero grad
    optimizer.zero_grad() # if not; going to exploding gradient
    
    # 4. Backpropagation
    loss.backward()
    
    # 5. Step Optimizer
    optimizer.step()
     
    ### Testing
    model_0.eval() # turns off different settings in the model not needed for evaluating/testing.

    with torch.inference_mode():# turns off gradient tracking & couple more things
        # 1. Forward Pass
        test_pred = model_0(X_test)
        
        # 2. Loss
        test_loss = loss_fn(test_pred, y_test)
        
    # Output
    if epoch % 10 == 0:
        epoch_count.append(epoch)
        train_loss_values.append(loss)
        test_loss_values.append(test_loss)
        
        print(f"Epoch : {epoch} | Loss : {loss} | Test Loss : {test_loss}")
        print(model_0.state_dict())
        print()
        
    
    

Epoch : 0 | Loss : 0.6860889196395874 | Test Loss : 0.7637526988983154
OrderedDict([('weights', tensor([0.8784])), ('bias', tensor([0.9050]))])

Epoch : 10 | Loss : 0.5708791017532349 | Test Loss : 0.6290428042411804
OrderedDict([('weights', tensor([0.8394])), ('bias', tensor([0.8050]))])

Epoch : 20 | Loss : 0.45566922426223755 | Test Loss : 0.4943329691886902
OrderedDict([('weights', tensor([0.8004])), ('bias', tensor([0.7050]))])

Epoch : 30 | Loss : 0.34045934677124023 | Test Loss : 0.35962313413619995
OrderedDict([('weights', tensor([0.7614])), ('bias', tensor([0.6050]))])

Epoch : 40 | Loss : 0.2252494841814041 | Test Loss : 0.2249133139848709
OrderedDict([('weights', tensor([0.7224])), ('bias', tensor([0.5050]))])

Epoch : 50 | Loss : 0.1100396141409874 | Test Loss : 0.09020347893238068
OrderedDict([('weights', tensor([0.6834])), ('bias', tensor([0.4050]))])

Epoch : 60 | Loss : 0.009724985808134079 | Test Loss : 0.020998019725084305
OrderedDict([('weights', tensor([0.6539])), (

In [23]:
with torch.inference_mode():
    y_preds_new = model_0(X_test)

In [24]:
# plot_prediction(predictions=y_preds_new)

In [25]:
train_loss_values

[tensor(0.6861, grad_fn=<MeanBackward0>),
 tensor(0.5709, grad_fn=<MeanBackward0>),
 tensor(0.4557, grad_fn=<MeanBackward0>),
 tensor(0.3405, grad_fn=<MeanBackward0>),
 tensor(0.2252, grad_fn=<MeanBackward0>),
 tensor(0.1100, grad_fn=<MeanBackward0>),
 tensor(0.0097, grad_fn=<MeanBackward0>),
 tensor(0.0062, grad_fn=<MeanBackward0>),
 tensor(0.0028, grad_fn=<MeanBackward0>),
 tensor(0.0071, grad_fn=<MeanBackward0>)]

In [26]:
torch.tensor(train_loss_values).numpy()

array([0.6860889 , 0.5708791 , 0.45566922, 0.34045935, 0.22524948,
       0.11003961, 0.00972499, 0.00621675, 0.00278832, 0.00709595],
      dtype=float32)

In [28]:
"""
plt.plot(epoch_count, torch.tensor(train_loss_values).numpy(), label = 'Train Loss')
plt.plot(epoch_count, torch.tensor(test_loss_values).numpy(), label = 'Test Loss')

plt.xlabel('Epochs')
plt.ylabel('Loss')

plt.title('Train and Test Loss Values')
plt.legend();
"""

"\nplt.plot(epoch_count, torch.tensor(train_loss_values).numpy(), label = 'Train Loss')\nplt.plot(epoch_count, torch.tensor(test_loss_values).numpy(), label = 'Test Loss')\n\nplt.xlabel('Epochs')\nplt.ylabel('Loss')\n\nplt.title('Train and Test Loss Values')\nplt.legend();\n"

- ### Save & Load The Model

3 main methods for saving/loading models in PyTorch :
- 1. `torch.save()` - save a PyTorch objects in python pickle format

- 2. `torch.load()` - load a saved PyTorch object.

- 3. `torch.nn.Module.load_state_dict()` - load a model's saved state dictionary.

In [32]:
### Recommended in Pytorch Documentation :

# save :
from pathlib import Path

# create directory
MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents=True, exist_ok=True)

# saving :
MODEL_NAME = '01_workflow_0.pth'
torch.save(model_0.state_dict(), f"{MODEL_PATH}/{MODEL_NAME}")

In [33]:
# load :
my_model_1 = LinearRegressionModel()
my_model_1.load_state_dict(torch.load(f"{MODEL_PATH}/{MODEL_NAME}"))

<All keys matched successfully>